# Setup

In [ ]:
!pip install tensorboardX --quiet
!pip install tensorboard --quiet

In [ ]:
# Install deps
! pip install fairseq sacrebleu sentencepiece --quiet

In [ ]:
# Install nvidia apex for multi-tensor support
# Ref: https://zhuanlan.zhihu.com/p/194176917
%cd /home/ubuntu/nmt
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

import fairseq
print(fairseq.utils.multi_tensor_l2norm_available)


## Downloads

In [ ]:
# FLoRes repo clone
%cd /home/ubuntu/nmt
!git clone https://github.com/facebookresearch/flores.git

In [ ]:
# get mBART pretrained model
%cd /home/ubuntu/nmt/flores/
!wget https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.cc25.v2.tar.gz
!tar -xzvf mbart.cc25.v2.tar.gz
!rm -rf mbart.cc25.v2.tar.gz

# Focal Loss setup
Note: Make sure to upload the focal_loss.py file to notebook home

In [ ]:
%cd /home/ubuntu/nmt/
!cp -rf focal_loss.py $(pip show fairseq | grep Location | cut -d' ' -f2)/fairseq/criterions/

# CodeEnv Setup

In [ ]:
SRC="si"
# PARENT_SRC="ta"
TGT="en"
BPE=1 #6 for si-en, 1 for all others
NUM_UPDATES=45000
SAVE_INTERVAL=15000

In [ ]:
%cd /home/ubuntu/nmt/
!cp -f download_data.sh ./flores/download_data.sh
!cp -f prepare_parallel.sh ./flores/prepare_parallel.sh
!cp -f prepare_reverse.sh ./flores/prepare_reverse.sh


In [ ]:
# get baseline model
!mkdir -p /home/ubuntu/nmt/flores/checkpoints
%cd /home/ubuntu/nmt/flores/checkpoints
!wget https://msainmtntu.s3-ap-southeast-1.amazonaws.com/checkpoints/"checkpoint_best_parallel_"$TGT"_"$SRC".pt"
# !mv "checkpoint_best_parallel_"$TGT"_"$SRC".pt" checkpoint_best.pt


# Data Setup

In [ ]:
# Training data
%cd /home/ubuntu/nmt/flores/
!bash download-data.sh --langs $SRC"-"$TGT

In [ ]:
%cd /home/ubuntu/nmt/flores/
! bash prepare_parallel.sh --src $SRC --tgt $TGT --bpe $BPE

# Fairseq-train

In [ ]:
#########################################################################################################
# Baseline
#########################################################################################################
# SI-EN
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/wiki_si_en_bpe5000/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang si --target-lang en \
  --criterion focal_loss \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file /home/ubuntu/nmt/flores/checkpoints/checkpoint_best_parallel_si_en.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir TensorboardFocalLossSIEN/ \
  --max-update 40000 \
  --checkpoint-suffix "_focal_loss_si_en" \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a mbart-moses-si-en-screenlog

In [ ]:
#########################################################################################################
# Baseline
#########################################################################################################
# NE-EN
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/wiki_ne_en_bpe5000/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang ne --target-lang en \
  --criterion focal_loss \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file /home/ubuntu/nmt/flores/checkpoints/checkpoint_best_parallel_ne_en.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir TensorboardFocalLossNEEN/ \
  --max-update 40000 \
  --checkpoint-suffix "_focal_loss_ne_en" \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a mbart-moses-ne-en-screenlog

In [ ]:
#########################################################################################################
# Baseline
#########################################################################################################
# PS-EN
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/wiki_ps_en_bpe5000/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang ps --target-lang en \
  --criterion focal_loss \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file /home/ubuntu/nmt/flores/checkpoints/checkpoint_best_parallel_ps_en.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir TensorboardFocalLossPSEN/ \
  --max-update 40000 \
  --checkpoint-suffix "_focal_loss_ps_en" \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a mbart-moses-ps-en-screenlog

In [ ]:
#########################################################################################################
# Baseline
#########################################################################################################
# KM-EN
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/wiki_km_en_bpe5000/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang km --target-lang en \
  --criterion focal_loss \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file /home/ubuntu/nmt/flores/checkpoints/checkpoint_best_parallel_km_en.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir TensorboardFocalLossKMEN/ \
  --max-update 50000 \
  --checkpoint-suffix "_focal_loss_km_en" \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a mbart-moses-km-en-screenlog


# Eval

In [ ]:
#Generate Sacrebleu
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
!fairseq-generate ./data-bin/"wiki_"$SRC"_"$TGT"_bpe5000/" \
    --source-lang $SRC --target-lang $TGT \
    --path /home/ubuntu/nmt/flores/checkpoints/"checkpoint_best_focal_loss_"$SRC"_"$TGT".pt" \
    --task translation_from_pretrained_bart \
    --lenpen 1.2 \
    --gen-subset test \
    --bpe 'sentencepiece' --sentencepiece-model mbart.cc25.v2/sentence.bpe.model \
    --scoring sacrebleu  --batch-size 32 --langs $langs > $SRC"_"$TGT"_bleu_w_bpe"

!tail -1 $SRC"_"$TGT"_bleu_w_bpe"


In [ ]:
# !cat $SRC"_"$TGT"_bleu_w_bpe" | grep -P "^D" |sort -V |cut -f 3- | sed 's/\[en_XX\]//g' > $SRC"_"$TGT".hyp"
# !cat $SRC"_"$TGT"_bleu_w_bpe" | grep -P "^T" |sort -V |cut -f 2- | sed 's/\[en_XX\]//g' > $SRC"_"$TGT".ref"
# import sacrebleu
# my_hyp_file = open($SRC"_"$TGT".hyp", "r")
# hyp = my_hyp_file.readlines()
# my_ref_file = open($SRC"_"$TGT".ref", "r")
# ref_tmp = my_ref_file.readlines()
# ref = [sent for sent in ref_tmp]
# ref =[ref]
# bleu = sacrebleu.corpus_bleu(hyp,ref)
# print(bleu.score)

# Housekeeping

In [ ]:
# Upload checkpoint
!aws s3 cp /home/ubuntu/nmt/flores/checkpoints/"checkpoint_best_focal_loss_"$SRC"_"$TGT".pt" s3://msainmtntu/checkpoints/"checkpoint_best_focal_loss_"$SRC"_"$TGT".pt"
!rm -rf /home/ubuntu/nmt/flores/checkpoints/*"0000_focal_loss_"$SRC"_"$TGT".pt"


In [ ]:
# Upload tensorboard
!aws s3 cp --recursive /home/ubuntu/nmt/flores/TensorboardChild$(echo "$SRC" | tr a-z A-Z)$(echo "$TGT" | tr a-z A-Z) s3://msainmtntu/Tensorboards/TensorboardChild$(echo "$SRC" | tr a-z A-Z)$(echo "$TGT" | tr a-z A-Z)


## Checks

In [ ]:
!ls -lh /home/ubuntu/nmt/flores/checkpoints/

In [ ]:
!tail -10 /home/ubuntu/nmt/flores/"mbart-moses-"$SRC"-"$TGT"-screenlog"